# Neural Crest Migration Area Analysis
This code was used to neural crest migration areas measured from whole mount images.

Required inputs for this script:

1. .csv file for containing source data for each embryo documenting the neural crest migration area on experimental and contralateral control sides.

Script prepared by Mike Piacentino, March 2021

In [9]:
# Import data handling and analysis packages
import os
import glob
import pandas as pd
from scipy import stats

# Import plotting packages
import iqplot
import bokeh.io
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

## Import Data

In [10]:
# Add analysis date here to apply to dataframe
path = os.path.abspath('')+'/raw_data_csvs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    (df['ExptDate'], df['Construct'], df['Dose'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag']) = zip(*df['Image'].map(lambda x: x.split('_')))
    df['Treatment'] = df['Construct']
    list_.append(df)

full_df = pd.concat(list_)
full_df = full_df.drop(['Label', 'Mean', 'IntDen', 'RawIntDen', 'delete1', 'delete2', 'delete3'], axis=1)
source_data = full_df
source_data.head()

,,Area,Image,ROI,ExptDate,Construct,Dose,Stains,Embryo,Somites,ImageMag,Treatment
0,1,53975.605,20180418_SMPD3gRNA1DNA_2ugul_Pax7NcadLaminin_E...,CntlArea,20180418,SMPD3gRNA1DNA,2ugul,Pax7NcadLaminin,Emb7,8ss,10x,SMPD3gRNA1DNA
1,2,34435.805,20180418_SMPD3gRNA1DNA_2ugul_Pax7NcadLaminin_E...,ExptArea,20180418,SMPD3gRNA1DNA,2ugul,Pax7NcadLaminin,Emb7,8ss,10x,SMPD3gRNA1DNA
0,1,80357.839,20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb5_9ss...,CntlArea,20171016,SMPD3gRNA1DNA,2ugul,Pax7Sox9,Emb5,9ss,10x,SMPD3gRNA1DNA
1,2,88925.670,20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb5_9ss...,ExptArea,20171016,SMPD3gRNA1DNA,2ugul,Pax7Sox9,Emb5,9ss,10x,SMPD3gRNA1DNA
0,1,83373.317,20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb9_9ss...,CntlArea,20171016,SMPD3gRNA1DNA,2ugul,Pax7Sox9,Emb9,9ss,10x,SMPD3gRNA1DNA


### Generate an Experimental/Control Area Ratio, and normalize individual areas

For normalization, all values are normalized to the mean of the control group.

In [11]:
# Get a list of treatments
treatment_list = source_data.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = source_data.groupby(['Treatment', 'Image', 'ROI', 'Construct'])['Image', 'ROI', 'Area'].mean()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
full_results = pd.DataFrame()
list_ = []
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    # Extract Cntl and Expt areas
    area_cntl = df_treatment.xs('CntlArea', level='ROI')
    area_expt = df_treatment.xs('ExptArea', level='ROI')

    # Generate Expt/Cntl Area Ratios
    area_ratios = pd.DataFrame(area_expt / area_cntl)
    area_ratios.columns = ['Expt/Cntl Area']

    # Normalize all migration area values to mean of control group
    norm_cntl = area_cntl/(float(area_cntl.mean()))
    norm_cntl.columns = ['Norm Cntl Area']
    norm_expt = area_expt/(float(area_cntl.mean()))
    norm_expt.columns = ['Norm Expt Area']
    norm_areas = pd.concat([norm_cntl, norm_expt], axis=1, sort=False)

    # Combine processed values into single dataframe and output as csv file analysis date + 'MigrationResults.csv'
    area_cntl = pd.DataFrame(area_cntl)
    area_cntl.columns = ['Cntl Area']
    area_expt = pd.DataFrame(area_expt)
    area_expt.columns = ['Expt Area']
    results = pd.concat([area_cntl, area_expt, area_ratios, norm_cntl, norm_expt], axis=1, sort=True)
    results.to_csv(treatment + '_MigrationResults.csv')              # Optional to save out results
    list_.append(results)

full_results = pd.concat(list_)
full_results.head()

,,Cntl Area,Expt Area,Expt/Cntl Area,Norm Cntl Area,Norm Expt Area
Image,Construct,,,,,
20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb2_7ss_10x,SMPD3gRNA1DNA,75291.301,51738.422,0.687177,0.886079,0.608893
20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb5_9ss_10x,SMPD3gRNA1DNA,80357.839,88925.670,1.106621,0.945706,1.046538
20171016_SMPD3gRNA1DNA_2ugul_Pax7Sox9_Emb9_9ss_10x,SMPD3gRNA1DNA,83373.317,77197.874,0.925930,0.981194,0.908517
20180418_SMPD3gRNA1DNA_2ugul_Pax7NcadLaminin_Emb1_8ss_10x,SMPD3gRNA1DNA,71843.804,51341.649,0.714629,0.845507,0.604224
20180418_SMPD3gRNA1DNA_2ugul_Pax7NcadLaminin_Emb5_9ss_10x,SMPD3gRNA1DNA,107900.300,65898.593,0.610736,1.269844,0.775540


## Plot and perform statistical analysis

In [31]:
data = full_results.reset_index()

# Build Stripbox plot
stripbox = iqplot.stripbox(
                    # Data to plot
                        data=data,
                        q='Expt/Cntl Area', q_axis='y',
                        cats=['Construct'], 

                    # Plot details
                        jitter=True, jitter_kwargs=dict(width=0.25),
                        marker_kwargs=dict(alpha=0.8, size=8,color='darkgray'),
                        box_kwargs=dict(line_color='black', line_width=1.5),
                        whisker_kwargs=dict(line_color='black', line_width=1.5),
                        median_kwargs=dict(line_color='black', line_width=2),
                        frame_width=150, frame_height=300,

#                     # Plot customizations
#                         order=targets,
                        y_range=(0,1.5),
                        y_axis_label='NC Migratory Area (Expt/Cntl)',
                        x_axis_label='Construct',
                        show_legend=False,
)

# Final customizations
stripbox.axis.axis_label_text_font_size = '16px'
stripbox.axis.major_label_text_font_size = '16px'
stripbox.axis.axis_label_text_font_style = 'bold'
stripbox.xaxis.major_label_text_font_style = 'italic'

# View plot
show(stripbox)

In [32]:
################### Isolate data for analysis ###################
# Annotate data further to plot 
cntl_construct = 'Control gRNA'
expt_construct = 'nSMase2 gRNA'
construct = 'SMPD3gRNA1DNA'


# Pull out only cells and treaments of interest, and rename ROIs with the appropriate constructs
data=full_results.reset_index()
data=data.loc[data['Construct'] == construct]
data=data.filter(['Image', 'Norm Cntl Area', 'Norm Expt Area'])
data=data.melt(id_vars=['Image'], var_name='ROI', value_name='Norm Area')
data.replace(to_replace = {'Norm Cntl Area': cntl_construct, 'Norm Expt Area': expt_construct}, inplace=True)

################### Plot as strip plot ###################
# Plot as strip plot
p1 = iqplot.strip(data=data
                ,q='Norm Area', q_axis='y'
                ,cats=['ROI'], parcoord_column='Image'
                ,y_range=(0,1.5)
                ,frame_height = 300, frame_width = 150
                ,y_axis_label= 'Normalized Migration Area'
#                 ,color_column='Image'
                ,marker_kwargs=dict(size=5,color='black')
                ,parcoord_kwargs=dict(line_width=1,color='gray')
#                 ,show_legend=True
              )

# p1.axis.axis_label_text_font_style = 'bold italic'
p1.axis.axis_label_text_font_size = '14px'
p1.axis.major_label_text_font_size = '12px'
p1.axis.axis_label_text_font_style = 'normal'
p1.xaxis.major_label_orientation = 7

show(row(p1))

################### Perform statistical analysis ###################

# Perform Paired t test 
cntl = data.loc[data['ROI'] == cntl_construct]['Norm Area']
expt = data.loc[data['ROI'] == expt_construct]['Norm Area']
ttest = stats.ttest_rel(cntl,expt)

# Display test results
print('Paired t-test results: \n\t\t statistic=' + str(ttest[0]) + 
    '\n\t\t p-value=' + str(ttest[1]))

Paired t-test results: 
		 statistic=3.6874920177547708
		 p-value=0.0077823157251296895
